In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import PyPDF2
import re
import json
import os

In [3]:
#Separar las palabras que solo pueden contener letras (nombres, localidades...)
#Separar las palabras que solo pueden contener numeros (cp, importes...)
#Separar las palabras que terminan o empiezan por una letra o numero (dni, cif)

In [6]:
train_path = 'training/'

In [8]:
json_paths = [train_path + file for file in os.listdir(train_path) if file.endswith('.json')]
pdf_paths = [train_path + file for file in os.listdir(train_path) if file.endswith('.pdf')]

In [10]:
json_files = {}
for json_path in json_paths:
    bill_id = json_path.removeprefix(train_path + 'factura_').removesuffix('.json')
    bill_id = int(bill_id)
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[bill_id] = data

In [12]:
bills_id = []
for data in json_files.values():
    bills_id.append(data['número_factura'])

In [14]:
pd.DataFrame({'bill_id':bills_id, 'len_bill': [len(bill) for bill in bills_id]})['len_bill'].value_counts()

len_bill
11    502
12    498
Name: count, dtype: int64

In [ ]:
json_files = {}
pdf_files = {}
for json_path, pdf_path in zip(json_paths, pdf_paths):
    json_bill_id = json_path.removeprefix(train_path + 'factura_').removesuffix('.json')
    json_bill_id = int(json_bill_id)
    
    pdf_bill_id = pdf_path.removeprefix(train_path + 'factura_').removesuffix('.pdf')
    pdf_bill_id = int(pdf_bill_id)
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[json_bill_id] = data
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
    
        # Obtener el número de páginas
        num_pages = len(reader.pages)

        # Extraer texto de todas las páginas
        all_text = ""
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()
        all_text = all_text.replace('\n','')
    pdf_files[pdf_bill_id] = all_text

In [253]:
len(pdf_files)

626

In [303]:
re_dni = r"\b\d{8}[A-Z]\b"
re_cif_national = r"\b[A-Z]\d{8}\b"
re_cif_foreign = r"\b[A-Z]{2}\d{6,8}\b"

In [305]:
bill_dni_cif = {}
for bill_id, text in pdf_files.items():
    dni = set(re.findall(re_dni, text))
    dni = list(dni)[0] if len(dni) == 1 else None
    
    cif = set(re.findall(re_cif_national, text))
    
    if len(cif) == 1:
        cif = list(cif)[0]
    else:
        cif = set(re.findall(re_cif_foreign, text))
        cif = list(cif)[0] if len(cif) == 1 else None
    
    bill_dni_cif[bill_id] = [dni, cif]

In [306]:
preds_dic = {}
for i in range(1000):
    pred_dni, pred_cif = bill_dni_cif[i]
    
    dni = json_files[i]['dni_cliente']
    cif = json_files[i]['cif_comercializadora']
    
    preds_dic[i] = [bool(pred_dni == dni), bool(pred_cif == cif)]

In [308]:
df = pd.DataFrame(preds_dic.values(), columns = ['dni', 'cif'])

In [311]:
len([data[1] for data in bill_dni_cif.values() if data[1] == None])

65

In [313]:
sum(df['cif'] == False)

65

In [315]:
df[df['cif'] == False].index

Index([ 46,  90,  99, 109, 124, 139, 142, 148, 164, 201, 240, 283, 296, 367,
       377, 386, 405, 413, 417, 426, 433, 436, 454, 458, 477, 507, 518, 521,
       527, 555, 561, 571, 589, 594, 595, 596, 605, 631, 661, 673, 692, 702,
       713, 724, 728, 742, 765, 774, 780, 789, 796, 855, 861, 874, 884, 900,
       903, 908, 917, 937, 962, 964, 969, 970, 987],
      dtype='int64')